In [1]:
!pip install ortools


     |████████████████████████████████| 14.4MB 234kB/s 
     |████████████████████████████████| 1.0MB 25.7MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


tutaj przypadkowe dystanse, pickup/delivery points i demands

In [41]:
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


n=50 #num of clients
nv=5 #num of vehicles
vc=[3]*nv #capacities

N=[i for i in range(1,n+1)] #clients
V=[0]+N #verticles with central point
x=np.random.rand(len(V))*100
y=np.random.rand(len(V))*100

p=[(i,j) for i in V for j in V ] #all posible routes  
d={(i,j): int(np.hypot([x[i]-x[j]],[y[i]-y[j]])[0]) for (i,j) in p}

distmat=list()

for i in range(n+1):
  distances=list()
  for j in range(n+1):
    distances.append(d[(i,j)])
  distmat.append(distances)



indexes=np.arange(1,n+1)
np.random.shuffle(indexes) #randomize indexes
pickups=indexes[0:int((n+1)/2)] #cut indexes in half to create random pickup and delivery points
deliveries=indexes[int((n+1)/2):(n+1)]

pnd=list()
for i in range(len(pickups)):
  pnd.append([pickups[i],deliveries[i]])



demands=np.zeros(len(pickups)*2+1)

for i in range(len(pickups)):
  demands[pickups[i]]+=np.random.randint(1,4)
  demands[deliveries[i]]+=(demands[pickups[i]]*(-1))

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distmat
    data['pickups_deliveries'] = pnd
    data['demands'] = demands
    data['vehicle_capacities'] = vc
    data['num_vehicles'] = nv
    data['depot'] = 0
    return data

In [42]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

main pickup n delivery z capacity constraints (chwilowo wyłączyłam uciekanie z lokalnych minimów, jest zakomentowane pod search parameters)

In [44]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    #search_parameters.local_search_metaheuristic = (
        #routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    #search_parameters.time_limit.FromSeconds(300)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

    print(solution)

if __name__ == '__main__':
    main()

Objective: 38811
Route for vehicle 0:
 0 Load(0.0) ->  10 Load(3.0) ->  14 Load(0.0) ->  13 Load(2.0) ->  11 Load(0.0) ->  32 Load(2.0) ->  30 Load(0.0) ->  0 Load(0.0)
Distance of the route: 358m
Load of the route: 0.0

Route for vehicle 1:
 0 Load(0.0) ->  33 Load(2.0) ->  20 Load(3.0) ->  3 Load(1.0) ->  41 Load(3.0) ->  5 Load(1.0) ->  50 Load(3.0) ->  47 Load(2.0) ->  46 Load(0.0) ->  21 Load(3.0) ->  29 Load(0.0) ->  25 Load(2.0) ->  36 Load(3.0) ->  40 Load(1.0) ->  31 Load(0.0) ->  0 Load(0.0)
Distance of the route: 353m
Load of the route: 0.0

Route for vehicle 2:
 0 Load(0.0) ->  22 Load(2.0) ->  16 Load(0.0) ->  48 Load(2.0) ->  15 Load(0.0) ->  0 Load(0.0)
Distance of the route: 369m
Load of the route: 0.0

Route for vehicle 3:
 0 Load(0.0) ->  6 Load(3.0) ->  19 Load(0.0) ->  27 Load(2.0) ->  34 Load(0.0) ->  1 Load(2.0) ->  12 Load(0.0) ->  9 Load(3.0) ->  7 Load(0.0) ->  43 Load(3.0) ->  35 Load(0.0) ->  45 Load(3.0) ->  28 Load(0.0) ->  0 Load(0.0)
Distance of the route